#**基础库**

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import print_function, division
import tensorflow as tf
from sklearn.utils import shuffle
import tifffile as tiff
from tensorflow import keras as K
import tensorflow.keras.layers as L
import numpy as np
import os
import time
import h5py
import argparse 
import random
import cv2
from tqdm import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LambdaCallback
# from keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from __future__ import print_function, division
from tensorflow.keras.layers import *
# from keras.layers import Dense,Dropout
from sklearn.utils import shuffle
# from keras.layers.core import Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
# from tensorflow.keras.layers.core import Lambda
from tensorflow.keras import backend as Kb
from collections import Counter
# from utils_1 import *

#基础部分

In [ ]:
def tf_flatten(a):
  """Flatten tensor"""
  return tf.reshape(a, [-1])


def tf_repeat(a, repeats, axis=0):
  """TensorFlow version of np.repeat for 1D"""
  # https://github.com/tensorflow/tensorflow/issues/8521
  assert len(a.get_shape()) == 1

  a = tf.expand_dims(a, -1)
  a = tf.tile(a, [1, repeats])
  a = tf_flatten(a)
  return a


def tf_repeat_2d(a, repeats):
  """Tensorflow version of np.repeat for 2D"""

  assert len(a.get_shape()) == 2
  a = tf.expand_dims(a, 0)
  a = tf.tile(a, [repeats, 1, 1])
  return a


def tf_map_coordinates(input, coords, order=1):
  """Tensorflow verion of scipy.ndimage.map_coordinates

  Note that coords is transposed and only 2D is supported

  Parameters
  ----------
  input : tf.Tensor. shape = (s, s)
  coords : tf.Tensor. shape = (n_points, 2)
  """

  assert order == 1

  coords_lt = tf.cast(tf.floor(coords), 'int32')
  #coords_rb = tf.cast(tf.ceil(coords), 'int32')
  coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
  coords_lb = tf.stack([coords_lt[:, 0], coords_rb[:, 1]], axis=1)
  coords_rt = tf.stack([coords_rb[:, 0], coords_lt[:, 1]], axis=1)

  vals_lt = tf.gather_nd(input, coords_lt)
  vals_rb = tf.gather_nd(input, coords_rb)
  vals_lb = tf.gather_nd(input, coords_lb)
  vals_rt = tf.gather_nd(input, coords_rt)

  coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
  vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[:, 0]
  vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[:, 0]
  mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[:, 1]

  return mapped_vals


def sp_batch_map_coordinates(inputs, coords):
  """Reference implementation for batch_map_coordinates"""
  coords = coords.clip(0, inputs.shape[1] - 1)
  mapped_vals = np.array([
      sp_map_coordinates(input, coord.T, mode='nearest', order=1)
      for input, coord in zip(inputs, coords)
  ])
  return mapped_vals


def tf_batch_map_coordinates(input, coords, order=1):
  """Batch version of tf_map_coordinates

  Only supports 2D feature maps

  Parameters
  ----------
  input : tf.Tensor. shape = (b, s, s)
  coords : tf.Tensor. shape = (b, n_points, 2)

  Returns
  -------
  tf.Tensor. shape = (b, s, s)
  """

  input_shape = tf.shape(input)
  batch_size = input_shape[0]
  input_size = input_shape[1]
  n_coords = tf.shape(coords)[1]

  coords = tf.clip_by_value(coords, 0, tf.cast(input_size, 'float32') - 1)
  coords_lt = tf.cast(tf.floor(coords), 'int32')
  #coords_rb = tf.cast(tf.ceil(coords), 'int32')
  coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
  coords_lb = tf.stack([coords_lt[..., 0], coords_rb[..., 1]], axis=-1)
  coords_rt = tf.stack([coords_rb[..., 0], coords_lt[..., 1]], axis=-1)

  idx = tf_repeat(tf.range(batch_size), n_coords)

  def _get_vals_by_coords(input, coords):
      indices = tf.stack([
          idx, tf_flatten(coords[..., 0]), tf_flatten(coords[..., 1])
      ], axis=-1)
      vals = tf.gather_nd(input, indices)
      vals = tf.reshape(vals, (batch_size, n_coords))
      return vals

  vals_lt = _get_vals_by_coords(input, coords_lt)
  vals_rb = _get_vals_by_coords(input, coords_rb)
  vals_lb = _get_vals_by_coords(input, coords_lb)
  vals_rt = _get_vals_by_coords(input, coords_rt)

  coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
  vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[..., 0]
  vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[..., 0]
  mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[..., 1]

  return mapped_vals


def sp_batch_map_offsets(input, offsets):
  """Reference implementation for tf_batch_map_offsets"""

  batch_size = input.shape[0]
  input_size = input.shape[1]

  offsets = offsets.reshape(batch_size, -1, 2)
  grid = np.stack(np.mgrid[:input_size, :input_size], -1).reshape(-1, 2)
  grid = np.repeat([grid], batch_size, axis=0)
  coords = offsets + grid
  coords = coords.clip(0, input_size - 1)

  mapped_vals = sp_batch_map_coordinates(input, coords)
  return mapped_vals


def tf_batch_map_offsets(input, offsets, order=1):
  """Batch map offsets into input

  Parameters
  ---------
  input : tf.Tensor. shape = (b, s, s)
  offsets: tf.Tensor. shape = (b, s, s, 2)

  Returns
  -------
  tf.Tensor. shape = (b, s, s)
  """

  input_shape = tf.shape(input)
  batch_size = input_shape[0]
  input_size = input_shape[1]

  offsets = tf.reshape(offsets, (batch_size, -1, 2))
  grid = tf.meshgrid(
      tf.range(input_size), tf.range(input_size), indexing='ij'
  )
  grid = tf.stack(grid, axis=-1)
  grid = tf.cast(grid, 'float32')
  grid = tf.reshape(grid, (-1, 2))
  grid = tf_repeat_2d(grid, batch_size)
  coords = offsets + grid

  mapped_vals = tf_batch_map_coordinates(input, coords)
  return mapped_vals
def BN_LeakyReLU(input):
    
  norm = L.BatchNormalization(axis=-1)(input)
  output = L.LeakyReLU(alpha=0.2)(norm)
  
  return output
class ConvOffset2D(Conv2D):
  """ConvOffset2D

  Convolutional layer responsible for learning the 2D offsets and output the
  deformed feature map using bilinear interpolation

  Note that this layer does not perform convolution on the deformed feature
  map. See get_deform_cnn in cnn.py for usage
  """

  def __init__(self, filters, init_normal_stddev=0.01, **kwargs):
    """Init

    Parameters
    ----------
    filters : int
        Number of channel of the input feature map
    init_normal_stddev : float
        Normal kernel initialization
    **kwargs:
        Pass to superclass. See Con2D layer in Keras
    """

    self.filters = filters
    super(ConvOffset2D, self).__init__(
        self.filters * 2, (3, 3), padding='same', use_bias=False,
        kernel_initializer=K.initializers.RandomNormal(0, init_normal_stddev),
        **kwargs
    )

  def call(self, x):
    """Return the deformed featured map"""
    x_shape = x.get_shape()
    offsets = super(ConvOffset2D, self).call(x)

    # offsets: (b*c, h, w, 2)
    offsets = self._to_bc_h_w_2(offsets, x_shape)

    # x: (b*c, h, w)
    x = self._to_bc_h_w(x, x_shape)

    # X_offset: (b*c, h, w)
    x_offset = tf_batch_map_offsets(x, offsets)

    # x_offset: (b, h, w, c)
    x_offset = self._to_b_h_w_c(x_offset, x_shape)

    return x_offset

  def compute_output_shape(self, input_shape):
    """Output shape is the same as input shape

    Because this layer does only the deformation part
    """
    return input_shape

  @staticmethod
  def _to_bc_h_w_2(x, x_shape):
    """(b, h, w, 2c) -> (b*c, h, w, 2)"""
    x = tf.transpose(x, [0, 3, 1, 2])
    x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2]), 2))
    return x

  @staticmethod
  def _to_bc_h_w(x, x_shape):
    """(b, h, w, c) -> (b*c, h, w)"""
    x = tf.transpose(x, [0, 3, 1, 2])
    x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2])))
    return x

  @staticmethod
  def _to_b_h_w_c(x, x_shape):
    """(b*c, h, w) -> (b, h, w, c)"""
    x = tf.reshape(
        x, (-1, int(x_shape[3]), int(x_shape[1]), int(x_shape[2]))
    )
    x = tf.transpose(x, [0, 2, 3, 1])
    return x

#全局参数

In [ ]:
#全局参数
hchn = 12
r = 7
#标签样本数据npy
scale=2000#每次抽取的无标签数量构建伪样本池
NUM_CLASS = 7

#backbone



In [ ]:
######################################backbone部分######################################
def feature_extraction_CNN(input_shape, training_parameter= False,n_filters=64):
  # X_input=L.Input(input_shape)
  conv1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(input_shape)
  
  conv1 = BN_LeakyReLU(conv1)
  conv1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv1)
  conv1 = BN_LeakyReLU(conv1)
  
  pool1 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv1)
  # print(pool1)
  #可变形block A1
  # print(pool1)
  offset_conv2_1 = ConvOffset2D(2*n_filters)(pool1)
  
  conv2_1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_1)
  conv2_1 = BN_LeakyReLU(conv2_1)
  
  offset_conv2_2 = ConvOffset2D(n_filters)(conv2_1)
  conv2_2 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_2)
  conv2_2 = BN_LeakyReLU(conv2_2)
  
  offset_conv2_3 = ConvOffset2D(int(0.5*n_filters))(conv2_2)
  conv2_3 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_3)
  conv2_3 = BN_LeakyReLU(conv2_3)
  
  conv2_4 = L.concatenate([conv2_1, conv2_2, conv2_3], axis=-1)
  # print(conv2_4)
  conv2_5 = L.add([conv2_4, pool1])

  #可变形block A2
  offset_conv2_6 = ConvOffset2D(2*n_filters)(conv2_5)
  conv2_6 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_6)
  conv2_6 = BN_LeakyReLU(conv2_6)
  
  offset_conv2_7 = ConvOffset2D(n_filters)(conv2_6)
  conv2_7 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_7)
  conv2_7 = BN_LeakyReLU(conv2_7)
  
  offset_conv2_8 = ConvOffset2D(int(0.5*n_filters))(conv2_7)
  conv2_8 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_8)
  conv2_8 = BN_LeakyReLU(conv2_8)
  
  conv2_9 = L.concatenate([conv2_6, conv2_7, conv2_8], axis=-1)
  
  conv2_10 = L.add([conv2_9, conv2_5])
  
  
  #pool2 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv2_5)
  
  conv3 = L.Conv2D(4*n_filters, (3, 3), padding='valid', strides=(2, 2), kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv2_5)
  
  offset_conv3_1 = ConvOffset2D(4*n_filters)(conv3)
  
  conv3_1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_1)
  conv3_1 = BN_LeakyReLU(conv3_1)
  
  offset_conv3_2 = ConvOffset2D(2*n_filters)(conv3_1)
  conv3_2 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_2)
  conv3_2 = BN_LeakyReLU(conv3_2)
  offset_conv3_3 = ConvOffset2D(n_filters)(conv3_2)
  conv3_3 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_3)
  conv3_3 = BN_LeakyReLU(conv3_3)
  
  conv3_4 = L.concatenate([conv3_1, conv3_2, conv3_3], axis=-1)
  
  conv3_5 = L.add([conv3_4, conv3])

  #改了一下4*
  offset_conv3_6 = ConvOffset2D(n_filters)(conv3_2)
  
  conv3_6 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_6)
  conv3_6 = BN_LeakyReLU(conv3_6)
  
  offset_conv3_7 = ConvOffset2D(2*n_filters)(conv3_6)
  conv3_7 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_7)
  conv3_7 = BN_LeakyReLU(conv3_7)
  
  offset_conv3_8 = ConvOffset2D(n_filters)(conv3_7)
  conv3_8 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_8)
  conv3_8 = BN_LeakyReLU(conv3_8)
  
  conv3_9 = L.concatenate([conv3_6, conv3_7, conv3_8], axis=-1)
  
  conv3_10 = L.add([conv3_9, conv3_5])
  
  
  #pool3 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv3)
  #输出128维计算度量
  # print(conv3_10)
  conv4 = L.Conv2D(n_filters*2, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv3_10)
  # print(conv4)
  gap = L.GlobalAvgPool2D()(conv4)
  # print(gap)
  #增加功能
  X = Dense(1024,kernel_regularizer=regularizers.l2(0.01),name='dense_layer1')(gap)
  X1 = Dropout(rate=0.5)(X,training=training_parameter)#打开training模式，测试时也使用dropout
  X2 = Dense(256,kernel_regularizer=regularizers.l2(0.01),name='dense_layer2')(X1)
  X3 = Dropout(rate=0.5)(X2,training=training_parameter)
  spatial_result = X3
  print('输出向量维度：' + str(spatial_result.shape))
  return spatial_result

#🔺模型训练（全监督）

In [ ]:
#数据读取（自己替换）
X_train = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/label_x.npy')
Y_train = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/label_y.npy')
X_val = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/val_x.npy')
Y_val = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/val_y.npy')
X_test = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/test_x.npy')
Y_test = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/test_y.npy')
unlabeled_pool = np.load('/content/gdrive/MyDrive/CM/RSL/RSL_dataset/unlabel_x.npy')

In [ ]:
#查看数据集结构
# Counter(Y_test)

In [ ]:
def logistic_model(training_para):
  ksize = 2 * r + 1
  inputshape = L.Input((ksize,ksize,hchn))
  # inputshape = (ksize,ksize,hchn)
  #返回全局平均池化结果
  output_feature = feature_extraction_CNN(inputshape,training_parameter=training_para, n_filters=64)
  logits = L.Dense(NUM_CLASS+1, activation = 'softmax')(output_feature)
  model = K.models.Model(inputshape, logits)
  adam = K.optimizers.Adam(lr=1e-3,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01)
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['acc'])
  
  return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
def train_logistic(X,Y,X_val,Y_val):
  model = logistic_model(training_para=True)
  X_train, Y_train = shuffle(X, Y)
  
  model_checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/CM/RSL/CE_200_M5D12.h5', monitor='val_loss', verbose=1, 
                                       save_best_only=True, save_weights_only=True,period=10)

  early_stopping =EarlyStopping(monitor='val_loss', patience=20)
  # print('a')
  last_model = model.fit(x=X,y=Y,batch_size=50,epochs=200,shuffle=True,validation_data=(X_val, Y_val),
              callbacks=[model_checkpoint, early_stopping])

In [ ]:
Y_train=to_categorical(Y_train)
Y_val=to_categorical(Y_val)

In [ ]:
#训练
train_logistic(X_train,Y_train,X_val,Y_val)

#Softmax门限函数

In [ ]:
#判断每一个的softmax概率都大于阈值
def softmax_threshold(softmax_result,threshold = 0.9):
  max_index = np.argmax(softmax_result,axis=1)
  # print(max_index)
  count = 0
  for i in range(softmax_result.shape[0]):
    # print(i)
    # print(max_index[i])
    if softmax_result[i,max_index[i]] >= threshold:
      count += 1
  if count == softmax_result.shape[0]:
    return True
  else:
    return count

In [ ]:
def get_highsoftmax(model,unlabeled,scale,threshold = 0.9):
  temp_unlabeled,temp_index = sampling_unlabel(unlabeled,scale=scale)
  temp_predict = model.predict(temp_unlabeled)
  temp_predict_max = np.max(temp_predict,axis=1)
  #选出大于阈值的样本、和索引及伪标签
  temp_predict = temp_predict[temp_predict_max > threshold]
  temp_unlabeled = temp_unlabeled[temp_predict_max > threshold]
  temp_index = temp_index[temp_predict_max > threshold]
  t_plabel = np.argmax(temp_predict,axis=1)
  return temp_unlabeled,t_plabel,temp_index

  

#高抗噪门限函数


In [ ]:
def random_crop(image, min_ratio=0.6, max_ratio=1.0):

  h, w = image.shape[:2]
  
  ratio = random.random()
  
  scale = min_ratio + ratio * (max_ratio - min_ratio)
  
  new_h = int(h*scale)    
  new_w = int(w*scale)
  
  y = np.random.randint(0, h - new_h)    
  x = np.random.randint(0, w - new_w)
  
  image = image[y:y+new_h, x:x+new_w, :]
  
  return image
def pad(image,pad_shape=(15,15,12)):
  dis = pad_shape[0] - image.shape[0]
  w1 = random.randint(0,dis)
  w2 = dis - w1
  # print(w1,w2)
  nn = float(np.mean(image))
  # print(type(nn))
  pad_img = cv2.copyMakeBorder(image, w1, w2, w1, w2, cv2.BORDER_CONSTANT, value=(0,0,0))
  # pad_img = np.pad(image[:,:,0:11], ((w1,w2),(w1,w2)),'constant',constant_values=0)
  # print(image.shape)
  # print(pad_img.shape)
  return pad_img
def random_crop_pad(image):
  temp_img = random_crop(image,min_ratio=0.8,max_ratio=1.0)
  temp_img = pad(temp_img)
  return temp_img
def random_crop_resize(image):
  temp_img = random_crop(image,min_ratio=0.6,max_ratio=1.0)
  temp_img = cv2.resize(temp_img,(15,15))
  return temp_img

In [ ]:
#1样本池
def sampling_unlabel(unlabeled_X,scale):
  pe = np.random.permutation(unlabeled_X.shape[0])
  unlabeled_X =unlabeled_X[pe]
  if len(unlabeled_X) <= scale:
    scale = len(unlabeled_X)
  unlabeled_X = unlabeled_X[0:scale,:]
  return unlabeled_X, pe[0:scale]
#2mixup给伪标签
def mixup_supervised(unlabeled,model,scale):
  temp_unlabeled,temp_index=sampling_unlabel(unlabeled,scale)
  # for i in range(len(temp_unlabeled)):
  plabel_all = []
  acc_all = []
  last_unlabeled = []
  last_index = []
  print('\n数据增强中……')
  for i in tqdm(range(len(temp_unlabeled)),position=0):
    # data_enhanced=np.zeros((7,15,15,12))
    #减少增强次数
    pseudo_label = []
    data_enhanced=np.zeros((10,15,15,12))
    data_enhanced[0,]=temp_unlabeled[i,]
    #水平垂直翻转
    data_enhanced[1,]=np.flip(temp_unlabeled[i,],axis=0)
    data_enhanced[2,]=np.flip(temp_unlabeled[i,],axis=1)
    #180、270度旋转
    data_enhanced[3,]=np.rot90(temp_unlabeled[i,],k=2)
    data_enhanced[4,]=np.rot90(temp_unlabeled[i,],k=3)
    #增加高斯噪声(轻噪，重噪)
    data_enhanced[5,] = cv2.GaussianBlur(temp_unlabeled[i,], (3,3),sigmaX=5,sigmaY=5)
    data_enhanced[6,] = cv2.GaussianBlur(temp_unlabeled[i,], (3,3),sigmaX=0.5,sigmaY=0.5)
    #随机裁剪缩放
    data_enhanced[7,] = random_crop_resize(temp_unlabeled[i,])
    #随机裁剪
    data_enhanced[8,] = random_crop_pad(temp_unlabeled[i,])
    data_enhanced[9,] = random_crop_pad(temp_unlabeled[i,])
    ##############################显示
    # plt.figure()
    # for j in range(len(data_enhanced)):
    #   plt.subplot(1,10,j+1)
    #   plt.imshow(data_enhanced[j,:,:,0])
    # plt.show()
    #增强结果的embed
    enhance_pred=model.predict(data_enhanced)
    #计算度量获取伪标签
    if softmax_threshold(enhance_pred,threshold=0.5) == True:
      pseudo_label = np.argmax(enhance_pred,axis=1)
      # print('都大于')
      counts = np.bincount(pseudo_label)
      plabel= np.argmax(counts)
      acc=counts[plabel]/len(pseudo_label)
      plabel_all.append(plabel)
      acc_all.append(acc)
      last_unlabeled.append(temp_unlabeled[i,])
      last_index.append(temp_index[i,])
    # else:
    #   temp_unlabeled=np.delete(temp_unlabeled,i,axis=0)
    #   temp_index=np.delete(temp_index,i)
  plabel_all = np.asarray(plabel_all,dtype=int)
  acc_all = np.asarray(acc_all,dtype=float)
  last_unlabeled = np.asarray(last_unlabeled)
  last_index = np.asarray(last_index)
  return last_unlabeled,last_index,plabel_all,acc_all

#低不确定性门限函数

In [ ]:
def get_uncertainty(h_unlabeled,t_label,h_index,model,uthreshold=0.1,times=100):
  uncertain_label = []
  all_uncertain_label = []
  print('\n不确定性计算中……')
  for i in tqdm(range(times),position=0):
    templabel = model.predict(h_unlabeled)
    templabel = np.argmax(templabel,axis=1)
    uncertain_label.append(templabel)
  uncertain_label = np.asarray(uncertain_label)
  col_var = np.var(uncertain_label,axis=0)
  lu_unlabeled = h_unlabeled[col_var < uthreshold]
  lu_index = h_index[col_var < uthreshold]
  lu_label = t_label[col_var < uthreshold]
  return lu_unlabeled,lu_label,lu_index

#精度评价与混淆矩阵画图


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, cohen_kappa_score
def get_OA(cm):
    
    total = cm.sum()
    
    diag_arr = np.diagonal(cm, offset=0)
    
    correct = diag_arr.sum()
    
    OA = (correct/total) * 100
    
    return OA

def get_cm_oa_kappa(Y_test, Y_pred):
    Y_pred = np.argmax(Y_pred, axis=1)
    
    Y_test = Y_test.tolist()
    Y_pred = Y_pred.tolist()
  
  
    cm = confusion_matrix(Y_test,Y_pred)
    
    oa = get_OA(cm)
    
    kappa = cohen_kappa_score( Y_test,Y_pred)
    
    
    return cm, oa, kappa

def load_npz(npz_path):
    
    npz_arr = np.load(npz_path)
    _files = npz_arr.files
    
    return npz_arr[_files[0]]

import matplotlib.pyplot as pl
from sklearn import metrics
# 相关库

def plot_matrix(y_test, y_pred, labels_name, title=None, thresh=0.8, axis_labels=None):
  Y_pred = np.argmax(y_pred, axis=1)
  
  y_true = y_test.tolist()
  Y_pred = Y_pred.tolist()
# 利用sklearn中的函数生成混淆矩阵并归一化
  cm = confusion_matrix(y_true, Y_pred)  # 生成混淆矩阵 
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 归一化

# 画图，如果希望改变颜色风格，可以改变此部分的cmap=pl.get_cmap('Blues')处
  pl.imshow(cm, interpolation='nearest', cmap=pl.get_cmap('Blues'))
  pl.colorbar()  # 绘制图例
  pl.title(title, fontsize=15,fontfamily='SimHei')
# 图像标题
  if title is not None:
      pl.title(title)
# 绘制坐标
  num_local = np.array(range(len(labels_name)))
  if axis_labels is None:
      axis_labels = labels_name
  pl.xticks(num_local, axis_labels, rotation=45, fontsize=10)  # 将标签印在x轴坐标上， 并倾斜45度
  pl.yticks(num_local, axis_labels, fontsize=10)  # 将标签印在y轴坐标上
  pl.ylabel('True label',fontsize=12,verticalalignment='center')
  pl.xlabel('Predicted label',fontsize=12,verticalalignment='center')
  

# 将百分比打印在相应的格子内，大于thresh的用白字，小于的用黑字
  for i in range(np.shape(cm)[0]):
      for j in range(np.shape(cm)[1]):
          if int(cm[i][j] * 100 + 0.5) > 0:
              pl.text(j, i, format(int(cm[i][j] * 100), 'd') + '%',
                      ha="center", va="center",
                      color="white" if cm[i][j] > thresh else "black")  # 如果要更改颜色风格，需要同时更改此行
# 显示
  # pl.figure(dpi=1000) 
  pl.savefig('/content/gdrive/MyDrive/CM/RSL/paperimage/cm_2.png',dpi=1000,bbox_inches = 'tight')
  # pl.savefig('/content/gdrive/MyDrive/CM/RSL/paperimage/cm.png')
  pl.show()
 

In [ ]:
labels = ['farmland', 'forest', 'grassland', 'water', 'built-up', 'bare land', 'others']

In [ ]:
model1 = import_model('/content/gdrive/MyDrive/CM/RSL/CE_highsoftmx_highmix_lowups4.h5',train_para = False)

In [ ]:
Y_pred = model1.predict(X_test, verbose=1)
# print(Y_test)

650/650 [==============================] - 6s 9ms/sample


In [ ]:
cm, oa, kappa = get_cm_oa_kappa(Y_test, Y_pred)
print('acc: {:.2f}%  Kappa: {:.4f}'.format(oa,kappa))
print('acc: {:2f}%'.format(oa))
print(cm)

acc: 91.23%  Kappa: 0.8970
acc: 91.230769%
[[99  0  0  0  0  1  0]
 [ 1 86 12  1  0  0  0]
 [ 1  5 90  0  0  4  0]
 [ 0  0  0 94  0  6  0]
 [ 0  0  3  0 86 11  0]
 [ 0  0  3  0  1 96  0]
 [ 0  1  5  1  0  1 42]]


In [ ]:
plot_matrix(Y_test ,Y_pred, [0,1,2,3,4,5,6], title='Confusion Matrix',
            axis_labels=labels)


findfont: Font family ['SimHei'] not found. Falling back to DejaVu Sans.


#半监督函数

In [ ]:
def delete_unlabeled(u_pool,d_index):
  # for i in d_index:
  d_pool = np.delete(u_pool,d_index,axis=0)
  return d_pool


In [ ]:
def import_model(weightpath,train_para):
  ksize = 2 * r + 1
  inputshape = L.Input((ksize,ksize,hchn))
  output_feature = feature_extraction_CNN(inputshape,training_parameter=train_para, n_filters=64)
  logits = L.Dense(NUM_CLASS+1, activation = 'softmax')(output_feature)
  model = K.models.Model(inputshape, logits)
  adam = K.optimizers.Adam(lr=1e-4,beta_1=0.9, beta_2=0.999, epsilon=None,decay=0.01)
  # adam = K.optimizers.Adam(lr=1e-6)
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['acc'])
  model.load_weights(weightpath)
  return model

In [ ]:
# #限定增加伪标签样本个数
def sampling_unlabel(unlabeled_X,scale):
  pe = np.random.permutation(unlabeled_X.shape[0])
  unlabeled_X =unlabeled_X[pe]
  if len(unlabeled_X) <= scale:
    scale = len(unlabeled_X)
  unlabeled_X = unlabeled_X[0:scale,:]
  return unlabeled_X, pe[0:scale]
def limited_pseudo_label(sample,plabel,index,lnumber=50):
  """
  lnumber:限制的单类最大个数
  """
  #逐类确认样本个数
  least_num = Counter(plabel).most_common()[-1][1]
  # print(least_num)
  flag = 0
  for i in range(1,8):
    ti = index[plabel == i]
    ts = sample[plabel == i]
    tl = plabel[plabel == i]
    # print(tl)
    if len(ti) >= lnumber:
      randnum = random.sample(range(len(ti)),lnumber)
    else:
      randnum = [j for j in range(len(ti))]
    # print(randnum)
    ti = ti[randnum]
    ts = ts[randnum,:]
    tl = tl[randnum]
    if flag == 0:
      last_sample = ts
      last_label = tl
      last_index = ti
      flag += 1
    else:
      last_sample = np.append(last_sample,ts,axis=0)
      last_label = np.append(last_label,tl)
      last_index = np.append(last_index,ti)
  return last_sample,last_label,last_index

#softmax高置信度+高抗噪性+UPS（三门控半监督）

In [ ]:
def train_hs_hm_lups(X,Y,X_val,Y_val,unlabeled_pool):
  ####设置参数
  num_times = 5#抽取num_times次，每次epoch轮
  X_train, Y_train = shuffle(X, Y)
  model_checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/CM/RSL/CE_highsoftmx_highmix_lowups6.h5', monitor='val_acc', verbose=1, 
                                       save_best_only=True, save_weights_only=True,period=3)
  early_stopping =EarlyStopping(monitor='val_acc', patience=20)
####载入初始权重，训练模式
 
  last_X = X
  last_Y = Y
  flag = 0

  for i in tqdm(range(num_times),position=0):
    #先选出高置信度
    mmodel = import_model('/content/gdrive/MyDrive/CM/RSL/CE_hightsoftmax&low_UPS.h5',train_para=False)
    temp_unlabel,temp_plabel,temp_index = get_highsoftmax(mmodel,unlabeled_pool,5000,threshold = 0.98)
    #再计算高抗噪性
    temp_unlabel,temp_plabel,temp_index = highmix_unlabeled_for_softmax(temp_unlabel,temp_index,temp_plabel,mmodel,threshold=0.9)
    #然后计算低不确定性，打开dropout
    mmodel = import_model('/content/gdrive/MyDrive/CM/RSL/CE_hightsoftmax&low_UPS.h5',train_para=True)
    temp_unlabel,temp_plabel,temp_index = unlabel2lowups(temp_unlabel,temp_index,temp_plabel,mmodel,uthreshold=0.1,times=100)
    print('初始训练集大小：',X.shape)
    # print(Counter(temp_plabel))
    temp_unlabel,temp_plabel,temp_index = limited_pseudo_label(temp_unlabel,temp_plabel,temp_index)#平衡抽样
    print('新增样本统计:',Counter(temp_plabel))
    unlabeled_pool = delete_unlabeled(unlabeled_pool,temp_index)#更新样本池
    temp_plabel = to_categorical(temp_plabel,8)#同类型
    # print(last_Y.shape)
    last_X = np.append(last_X,temp_unlabel,axis=0)
    last_Y = np.append(last_Y,temp_plabel,axis=0)
    print('增加厚训练集大小：',last_X.shape)
    # print(last_Y.shape)
    mmodel.fit(x=last_X,y=last_Y,batch_size=50,epochs=100,shuffle=True,validation_data=(X_val, Y_val),
                        callbacks=[model_checkpoint, early_stopping])
    flag += 1

In [ ]:
Y_train=to_categorical(Y_train)
Y_val=to_categorical(Y_val)

In [ ]:
train_hs_hm_lups(X_train,Y_train,X_val,Y_val,unlabeled_pool)